Some exploratory work on the dataset.  

In [1]:
import sqlite3
import numpy as np
import pandas as pd

In [2]:
#Take a look at all the fires.  This may take some time.
df = pd.read_csv('20k_data.csv')
df.shape

(20000, 40)

In [3]:
df.head()

,Unnamed: 0,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1026201,1026202,1155826,TFS_NC_204819,NONFED,ST-NCNCS,ST/C&L,USNCNCS,North Carolina Forest Service,NCNCS312,...,A,35.565000,-81.175000,14.0,MISSING/NOT SPECIFIED,NC,NaN,NaN,NaN,b'\x00\x01\xad\x10\x00\x0003333KT\xc0\xc0\x1e\...
1,980661,980662,1109412,TFS_FL_64661,NONFED,ST-FLFLS,ST/C&L,USFLFLS,Florida Forest Service,FLFLS16,...,B,27.240000,-80.280000,14.0,MISSING/NOT SPECIFIED,FL,Martin,85.0,Martin,b'\x00\x01\xad\x10\x00\x00P\xb8\x1e\x85\xeb\x1...
2,676917,676918,739922,SFO-WI058402192,NONFED,ST-NASF,ST/C&L,USWIWIS,Wisconsin Department of Natural Resources,WI Woodruff,...,A,45.593872,-89.263800,14.0,MISSING/NOT SPECIFIED,WI,NaN,NaN,NaN,b'\x00\x01\xad\x10\x00\x00\x08\xcc\xf9\x18\xe2...
3,1405562,1405563,19101556,SFO-GA-EAR-16-9/29/1993-1730,NONFED,ST-GAGAS,ST/C&L,USGAGAS,Georgia Forestry Commission,GAGAS,...,B,31.258900,-84.815600,14.0,MISSING/NOT SPECIFIED,GA,Early,99.0,Early,b'\x00\x01\xad\x10\x00\x00\x84\xa7W\xca24U\xc0...
4,1568120,1568121,201434414,FS-1502869,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,...,B,39.800278,-120.503056,14.0,MISSING/NOT SPECIFIED,CA,063,63.0,Plumas,b'\x00\x01\xad\x10\x00\x00T\x91\xf2\x0f2 ^\xc0...


In [4]:
df.columns.values

array(['Unnamed: 0', 'OBJECTID', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE',
       'SOURCE_SYSTEM', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID',
       'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT',
       'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID',
       'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME',
       'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID',
       'MTBS_FIRE_NAME', 'COMPLEX_NAME', 'FIRE_YEAR', 'DISCOVERY_DATE',
       'DISCOVERY_DOY', 'DISCOVERY_TIME', 'STAT_CAUSE_CODE',
       'STAT_CAUSE_DESCR', 'CONT_DATE', 'CONT_DOY', 'CONT_TIME',
       'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE',
       'OWNER_CODE', 'OWNER_DESCR', 'STATE', 'COUNTY', 'FIPS_CODE',
       'FIPS_NAME', 'Shape'], dtype=object)

Removing rows where discovery time and containment time are BOTH missing. However, this cuts the dataset in almost half. Better to use a technique to fill these NA values (either make a model to predict correct values, or stuff in the average as filler).

In [5]:
# relevant columns to be able to compute time to containment
time_cols = ['DISCOVERY_TIME', 'CONT_TIME', 'DISCOVERY_DATE', 'CONT_DATE']

In [6]:
df = df.dropna(subset=time_cols) # remove rows where both of these are missing
df.shape

(9560, 40)

For some reason the dates are not in the mm/dd/yyyy format as the kaggle thing claims.
According to: https://www.kaggle.com/rtatman/188-million-us-wildfires/discussion/39627 the dates are instead formatted in 'Julian time'. So these will need to be converted.

Supposedly in the SQL query you can select the dates appropriately using: `SELECT datetime(DISCOVERY_DATE) as DISCOVERY_DATE from Fires` which may automatically convert these into a better format (like mm/dd/yyyy).

In [7]:
from datetime import datetime

# assign this globally so make_time() can use this without recalculating each call
epoch = pd.to_datetime(0, unit='s').to_julian_date()
epoch

2440587.5

In [11]:
# convert the 4 columns in time_cols into datetime objects
# then calculate the timedelta 
def make_time(row):
    # pad leading 0s to front of time so 438 = 0438, etc
    disc_time = str(row['DISCOVERY_TIME']).zfill(4)
    disc_time = datetime.strptime(disc_time, '%H%M.%S').time()
    cont_time = str(row['CONT_TIME']).zfill(4)
    cont_time = datetime.strptime(cont_time, '%H%M.%S').time()
    # dates are formatted as Julian, convert to datetime
    disc_date = pd.to_datetime(row['DISCOVERY_DATE'] - epoch, unit='D')
    cont_date = pd.to_datetime(row['CONT_DATE'] - epoch, unit='D')
    # combine the time and the date
    disc = datetime.combine(disc_date, disc_time)
    cont = datetime.combine(cont_date, cont_time)
    delta = cont-disc 
    return delta

df['TIME_TO_CONT'] = df.apply(lambda row: make_time(row), axis=1)

In [9]:
df.head()

,Unnamed: 0,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,...,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape,TIME_TO_CONT
3,1405562,1405563,19101556,SFO-GA-EAR-16-9/29/1993-1730,NONFED,ST-GAGAS,ST/C&L,USGAGAS,Georgia Forestry Commission,GAGAS,...,31.258900,-84.815600,14.0,MISSING/NOT SPECIFIED,GA,Early,99.0,Early,b'\x00\x01\xad\x10\x00\x00\x84\xa7W\xca24U\xc0...,00:43:00
4,1568120,1568121,201434414,FS-1502869,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,...,39.800278,-120.503056,14.0,MISSING/NOT SPECIFIED,CA,063,63.0,Plumas,b'\x00\x01\xad\x10\x00\x00T\x91\xf2\x0f2 ^\xc0...,02:00:00
5,1060555,1060556,1272887,FS-368027,FED,FS-FIRESTAT,FS,USCABDF,San Bernardino National Forest,0512,...,34.207500,-117.270000,13.0,STATE OR PRIVATE,CA,NaN,NaN,NaN,b'\x00\x01\xad\x10\x00\x00\xe0z\x14\xaeGQ]\xc0...,01:37:00
6,1245952,1245953,1649103,SFO-GA-FY2001-Hancock-005,NONFED,ST-NASF,ST/C&L,USGAGAS,Georgia Forestry Commission,GAGAS,...,33.315300,-83.053600,8.0,PRIVATE,GA,Hancock,141.0,Hancock,b'\x00\x01\xad\x10\x00\x000\xc4\xb1.n\xc3T\xc0...,00:00:00
7,159742,159743,161311,FS-389259,FED,FS-FIRESTAT,FS,USWYBTF,Bridger-Teton National Forest,0403,...,43.439167,-110.998611,5.0,USFS,WY,NaN,NaN,NaN,b'\x00\x01\xad\x10\x00\x00\xd0\xb7\x92>\xe9\xb...,20:30:00


In [12]:
df['TIME_TO_CONT'].describe()

count                      9560
mean     1 days 01:05:56.943514
std      6 days 18:00:19.623580
min           -1 days +04:46:00
25%             0 days 00:28:00
50%             0 days 01:26:00
75%             0 days 04:56:00
max           213 days 00:14:00
Name: TIME_TO_CONT, dtype: object